In [110]:
from datetime import datetime
from tqdm import tqdm
import cv2
import time
from collections import namedtuple, Counter
import seaborn as sns; 
sns.set(style="ticks", color_codes=True)
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import precision_recall_fscore_support
root_logdir = "tf_logs"
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

#### Importing data

In [98]:
#opening up the traning labels
label_file = "labels/train.txt"
validation_file = "labels/val.txt"
training_labels = open(label_file,"r").read().split("\n")
validation_lables = open(validation_file, "r").read().split("\n")
labels ={
    0: "letter",
    1: "form",
    2: "email",
    3: "handwritten",
    4: "advertisement",
    5: "scientific report",
    6: "scientific publication",
    7: "specification",
    8: "file folder",
    9: "news article",
    10: "budget",
    11: "invoice",
    12: "presentation",
    13: "questionnaire",
    14: "resume",
    15: "memo"
}
a_labels = {}
a_lables_val = {}
rows_count = 754
col_count = 1000
valid_classes ={
    "2": 0,
    "4": 1,
    "10": 2,
    "14": 3
}

def get_labels(labels, label_store):
    for label in labels:
        if label.split("/")[0] in ["imagesa","imagesb","imagesc"]:
            label_split = label.split(" ")
            file_path = label_split[0]
            file_label = label_split[1]
            if file_label in valid_classes.keys():
                label_store[file_path] = file_label
    return label_store

a_labels = get_labels(training_labels, a_labels)
a_labels_val = get_labels(validation_lables, a_lables_val)
            

print("Total Number of files for training:", len(a_labels))

counter = {}
for key,value in a_labels.items():
    if value in counter.keys():
        counter[value] = counter[value] + 1
    else:
        counter[value] = 0
for k,v in counter.items():
    print("Label",labels[int(k)],":",v)

Total Number of files for training: 9315
Label advertisement : 2306
Label budget : 2324
Label resume : 2331
Label email : 2350


In [99]:
%matplotlib inline
image_size = 128, 128
def get_image(random_image, image_print =False):
    pil_im = Image.open("data/" + random_image, 'r')
    data = list(pil_im.getdata())
    if image_print:
        imshow(np.asarray(pil_im),interpolation='gaussian', cmap="gray")
        print("The Image Below Belongs to Class:", labels[int(a_labels[random_image])])
    if len(data) == 754000:
        modified_image = pil_im.resize(image_size, Image.ANTIALIAS)
        data = list(modified_image.getdata())
        return np.array(data, dtype=float)
    else:
        return False

In [100]:
random_image = get_image(random_image = list(a_labels.keys())[544])

#### CLEAN THE DATA

In [101]:
def one_hot_encode(x, n_classes):
    return np.eye(n_classes)[np.array(x).reshape(-1)]

def normalize(train_ds_X):
    norm_func = lambda t: t / 256
    return list(map(norm_func,train_ds_X))
    
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [102]:
train_X = []
train_y = []
val_X = []
val_y = []


for key,value in tqdm(a_labels.items(), desc="Processing Images For Training"):
    image_details = get_image(key)
    if type(image_details) != bool:
        train_X.append(image_details)
        train_y.append(valid_classes[value])
for key,value in tqdm(a_lables_val.items(), desc="Processing Images For Validation"):
    image_details = get_image(key)
    if type(image_details) != bool:
        val_X.append(image_details)
        val_y.append(valid_classes[value])

Processing Images For Validation: 100%|██████████| 1207/1207 [00:21<00:00, 54.94it/s]


In [104]:
train_X_norm = np.array(normalize(train_X))
train_Y_one_hot = one_hot_encode(train_y,4)
train_X_matrix = train_X_norm.reshape(-1, 128, 128,1)

val_X_norm = np.array(normalize(val_X))
val_Y_one_hot = one_hot_encode(val_y,4)
val_X_matrix = val_X_norm.reshape(-1, 128, 128,1)

#### CREATE THE CNN BASED NN

In [105]:
num_classes = 4
batch_size = 128
epochs = 100

In [111]:
model = Sequential()
model.add(Conv2D(32, (8, 8), padding='same',
                 input_shape=train_X_matrix.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (8, 8)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (6, 6), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (6, 6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#### TRAIN THE MODEL

In [112]:
train_X_matrix.shape

(6010, 128, 128, 1)

In [113]:
model.fit(train_X_matrix, train_Y_one_hot,
          batch_size=batch_size,
          epochs=epochs, validation_data=(val_X_matrix, val_Y_one_hot),callbacks=callbacks_list,
          verbose=1)

Train on 6010 samples, validate on 809 samples
Epoch 1/100
6010/6010 [==============================] - 7s 1ms/step - loss: 1.0565 - acc: 0.5474 - val_loss: 0.9800 - val_acc: 0.5439
Epoch 2/100
6010/6010 [==============================] - 6s 965us/step - loss: 0.7333 - acc: 0.7285 - val_loss: 0.6791 - val_acc: 0.7454
Epoch 3/100
6010/6010 [==============================] - 6s 978us/step - loss: 0.6464 - acc: 0.7587 - val_loss: 0.6281 - val_acc: 0.7577
Epoch 4/100
6010/6010 [==============================] - 6s 968us/step - loss: 0.6024 - acc: 0.7755 - val_loss: 0.5820 - val_acc: 0.7701
Epoch 5/100


6010/6010 [==============================] - 6s 970us/step - loss: 0.5579 - acc: 0.7920 - val_loss: 0.5901 - val_acc: 0.7614
Epoch 6/100
6010/6010 [==============================] - 6s 964us/step - loss: 0.5317 - acc: 0.8032 - val_loss: 0.5150 - val_acc: 0.7899
Epoch 7/100
6010/6010 [==============================] - 6s 964us/step - loss: 0.5128 - acc: 0.8106 - val_loss: 0.4730 - val_acc: 0.8269
Epoch 8/100
6010/6010 [==============================] - 6s 962us/step - loss: 0.4898 - acc: 0.8215 - val_loss: 0.4720 - val_acc: 0.8133
Epoch 9/100


6010/6010 [==============================] - 6s 957us/step - loss: 0.4623 - acc: 0.8346 - val_loss: 0.4235 - val_acc: 0.8529
Epoch 10/100
6010/6010 [==============================] - 6s 954us/step - loss: 0.4421 - acc: 0.8404 - val_loss: 0.5842 - val_acc: 0.7849
Epoch 11/100
6010/6010 [==============================] - 6s 959us/step - loss: 0.4279 - acc: 0.8471 - val_loss: 0.3871 - val_acc: 0.8665
Epoch 12/100
6010/6010 [==============================] - 6s 955us/step - loss: 0.4089 - acc: 0.8594 - val_loss: 0.4060 - val_acc: 0.8566
Epoch 13/100


6010/6010 [==============================] - 6s 954us/step - loss: 0.3888 - acc: 0.8672 - val_loss: 0.3972 - val_acc: 0.8616
Epoch 14/100
6010/6010 [==============================] - 6s 954us/step - loss: 0.3698 - acc: 0.8689 - val_loss: 0.3957 - val_acc: 0.8591
Epoch 15/100
6010/6010 [==============================] - 6s 957us/step - loss: 0.3473 - acc: 0.8735 - val_loss: 0.3831 - val_acc: 0.8640
Epoch 16/100
6010/6010 [==============================] - 6s 954us/step - loss: 0.3413 - acc: 0.8819 - val_loss: 0.4232 - val_acc: 0.8443
Epoch 17/100


6010/6010 [==============================] - 6s 961us/step - loss: 0.3327 - acc: 0.8822 - val_loss: 0.3356 - val_acc: 0.8813
Epoch 18/100
6010/6010 [==============================] - 6s 959us/step - loss: 0.3118 - acc: 0.8917 - val_loss: 0.3034 - val_acc: 0.8888
Epoch 19/100
6010/6010 [==============================] - 6s 957us/step - loss: 0.3091 - acc: 0.8905 - val_loss: 0.3692 - val_acc: 0.8727
Epoch 20/100
6010/6010 [==============================] - 6s 958us/step - loss: 0.2905 - acc: 0.8995 - val_loss: 0.2997 - val_acc: 0.8937
Epoch 21/100


...

6010/6010 [==============================] - 6s 955us/step - loss: 0.0237 - acc: 0.9928 - val_loss: 0.2643 - val_acc: 0.9456
Epoch 82/100
6010/6010 [==============================] - 6s 957us/step - loss: 0.0189 - acc: 0.9937 - val_loss: 0.2520 - val_acc: 0.9444
Epoch 83/100
6010/6010 [==============================] - 6s 958us/step - loss: 0.0263 - acc: 0.9913 - val_loss: 0.2527 - val_acc: 0.9481
Epoch 84/100
6010/6010 [==============================] - 6s 947us/step - loss: 0.0204 - acc: 0.9947 - val_loss: 0.2860 - val_acc: 0.9518
Epoch 85/100


6010/6010 [==============================] - 6s 949us/step - loss: 0.0215 - acc: 0.9937 - val_loss: 0.2659 - val_acc: 0.9506
Epoch 86/100
6010/6010 [==============================] - 6s 947us/step - loss: 0.0210 - acc: 0.9928 - val_loss: 0.2703 - val_acc: 0.9481
Epoch 87/100
6010/6010 [==============================] - 6s 947us/step - loss: 0.0168 - acc: 0.9942 - val_loss: 0.2649 - val_acc: 0.9530
Epoch 88/100
6010/6010 [==============================] - 6s 948us/step - loss: 0.0179 - acc: 0.9952 - val_loss: 0.2527 - val_acc: 0.9567
Epoch 89/100


6010/6010 [==============================] - 6s 945us/step - loss: 0.0173 - acc: 0.9953 - val_loss: 0.2669 - val_acc: 0.9431
Epoch 90/100
6010/6010 [==============================] - 6s 946us/step - loss: 0.0203 - acc: 0.9928 - val_loss: 0.2335 - val_acc: 0.9518
Epoch 91/100
6010/6010 [==============================] - 6s 945us/step - loss: 0.0167 - acc: 0.9947 - val_loss: 0.2880 - val_acc: 0.9345
Epoch 92/100
6010/6010 [==============================] - 6s 946us/step - loss: 0.0137 - acc: 0.9958 - val_loss: 0.3052 - val_acc: 0.9493
Epoch 93/100


6010/6010 [==============================] - 6s 946us/step - loss: 0.0156 - acc: 0.9955 - val_loss: 0.2796 - val_acc: 0.9407
Epoch 94/100
6010/6010 [==============================] - 6s 945us/step - loss: 0.0161 - acc: 0.9953 - val_loss: 0.2551 - val_acc: 0.9493
Epoch 95/100
6010/6010 [==============================] - 6s 946us/step - loss: 0.0144 - acc: 0.9962 - val_loss: 0.2537 - val_acc: 0.9506
Epoch 96/100
6010/6010 [==============================] - 6s 945us/step - loss: 0.0129 - acc: 0.9958 - val_loss: 0.2684 - val_acc: 0.9555
Epoch 97/100


6010/6010 [==============================] - 6s 943us/step - loss: 0.0148 - acc: 0.9953 - val_loss: 0.2765 - val_acc: 0.9518
Epoch 98/100
6010/6010 [==============================] - 6s 946us/step - loss: 0.0179 - acc: 0.9938 - val_loss: 0.2673 - val_acc: 0.9506
Epoch 99/100
6010/6010 [==============================] - 6s 945us/step - loss: 0.0098 - acc: 0.9972 - val_loss: 0.2785 - val_acc: 0.9530
Epoch 100/100
6010/6010 [==============================] - 6s 945us/step - loss: 0.0230 - acc: 0.9937 - val_loss: 0.2740 - val_acc: 0.9481


#### TEST THE MODEL